In [1]:
import sys
import os

# Check current working directory
current_dir = os.getcwd()
print(f"Current Directory: {current_dir}")

# Change to your project root directory
project_root = '/home/razvansavin/Proiecte/flexiai'
os.chdir(project_root)
print(f"Changed Directory to: {os.getcwd()}")

# Add project root directory to sys.path
sys.path.append(project_root)
print(f"Project root added to sys.path")

Current Directory: /home/razvansavin/Proiecte/flexiai/examples/Code examples
Changed Directory to: /home/razvansavin/Proiecte/flexiai
Project root added to sys.path


In [2]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from flexiai.core.flexiai_client import FlexiAI
from flexiai.config.logging_config import setup_logging


# Set up logging
setup_logging()

# Initialize FlexiAI
flexiai = FlexiAI()

# Create an embedding
text = "OpenAI provides powerful tools for developers."
try:
    embedding = flexiai.embedding_manager.create_embeddings(text)
    print("Embedding created successfully")
    print(f"{embedding[:20]}...")
except Exception as e:
    print(f"Error creating embedding: {str(e)}")


Current working directory: /home/razvansavin/Proiecte/flexiai
Log directory '/home/razvansavin/Proiecte/flexiai/logs' created/exists.
Embedding created successfully
[ 0.00171095 -0.00803227 -0.01134489 -0.01328466  0.00326994  0.00642376
 -0.00899532 -0.00340483  0.00771808 -0.04169128  0.02406948  0.02852274
  0.0284681  -0.02176088 -0.00404687  0.01622846  0.02009432 -0.0105321
  0.01150199  0.00212759]...


---
---
---

In [3]:
%pip install numpy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### 1. **Text Similarity and Search**

#### Example: Finding Similar Texts
You can use embeddings to find texts that are similar to a given input text. This is useful for search engines, recommendation systems, and clustering similar documents.


In [4]:
def cosine_similarity(vec1, vec2):
    """Calculate the cosine similarity between two vectors."""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Embedding for the input text
input_text = "OpenAI provides powerful tools for developers."
input_embedding = flexiai.embedding_manager.create_embeddings(input_text)

# Embeddings for a list of texts
texts = [
    "Developers use OpenAI tools for creating applications.",
    "AI is revolutionizing technology.",
    "Cooking recipes are easy to find online.",
]
embeddings = [flexiai.embedding_manager.create_embeddings(text) for text in texts]

# Find the most similar text
similarities = [cosine_similarity(input_embedding, emb) for emb in embeddings]
most_similar_index = np.argmax(similarities)
print(f"Most similar text: {texts[most_similar_index]}")


Most similar text: Developers use OpenAI tools for creating applications.


### 2. **Clustering**

#### Example: Clustering Texts
Clustering texts based on their embeddings can help in organizing and categorizing large datasets.

In [5]:
# Example: Clustering Texts
texts = [
    "OpenAI provides powerful tools for developers.",
    "Developers use OpenAI tools for creating applications.",
    "AI is revolutionizing technology.",
    "Cooking recipes are easy to find online.",
    "Artificial intelligence is a growing field.",
    "Food blogs are popular.",
    "Machine learning is a subset of AI.",
    "I love baking new recipes.",
    "AI tools help developers build software.",
    "Technology is advancing rapidly.",
    "Recipes for healthy eating are trending.",
    "AI is used in various industries.",
]

# Generate embeddings for texts
embeddings = [flexiai.embedding_manager.create_embeddings(text) for text in texts]

# Scale embeddings
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)

# Clustering with KMeans
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(scaled_embeddings)

# Print clustered texts
for i in range(num_clusters):
    cluster_texts = [texts[j] for j in range(len(texts)) if clusters[j] == i]
    print(f"Cluster {i}:")
    for text in cluster_texts:
        print(f" - {text}")


Cluster 0:
 - AI is revolutionizing technology.
 - Artificial intelligence is a growing field.
 - Machine learning is a subset of AI.
 - Technology is advancing rapidly.
 - AI is used in various industries.
Cluster 1:
 - Cooking recipes are easy to find online.
 - Food blogs are popular.
 - I love baking new recipes.
 - Recipes for healthy eating are trending.
Cluster 2:
 - OpenAI provides powerful tools for developers.
 - Developers use OpenAI tools for creating applications.
 - AI tools help developers build software.


### 3. **Semantic Search**

#### Example: Implementing Semantic Search
Semantic search uses embeddings to find relevant documents based on the meaning of the query rather than keyword matching.


In [6]:
# Example: Semantic Search
query = "How does AI impact technology?"
query_embedding = flexiai.embedding_manager.create_embeddings(query)

# Assuming `documents` is a list of document texts and `doc_embeddings` is a list of their embeddings
documents = [
    "AI is revolutionizing technology.",
    "Cooking recipes are easy to find online.",
    "Developers use OpenAI tools for creating applications.",
]
doc_embeddings = [flexiai.embedding_manager.create_embeddings(doc) for doc in documents]

# Find the most relevant document
similarities = [cosine_similarity(query_embedding, emb) for emb in doc_embeddings]
most_relevant_index = np.argmax(similarities)
print(f"Most relevant document: {documents[most_relevant_index]}")


Most relevant document: AI is revolutionizing technology.


### 4. **Text Classification**

#### Example: Classifying Texts
Embeddings can be used as features for text classification tasks such as sentiment analysis, topic categorization, etc.


In [7]:
# Example: Text Classification
texts = [
    "I love using OpenAI tools.",
    "AI is transforming the industry.",
    "I found a great recipe for pasta.",
    "Artificial intelligence is fascinating.",
]
labels = [1, 1, 0, 1]  # 1 for tech-related, 0 for non-tech-related

embeddings = [flexiai.embedding_manager.create_embeddings(text) for text in texts]

# Train a simple classifier
classifier = LogisticRegression()
classifier.fit(embeddings, labels)

# Classify a new text
new_text = "Developers are creating amazing applications with AI."
new_embedding = flexiai.embedding_manager.create_embeddings(new_text)
predicted_label = classifier.predict([new_embedding])
print(f"Predicted label: {'tech-related' if predicted_label[0] == 1 else 'non-tech-related'}")


Predicted label: tech-related


### 5. **Question Answering**

#### Example: Answering Questions with Context
Using embeddings, you can implement a basic question-answering system that retrieves the most relevant answer from a given context.

In [8]:
# Example: Question Answering
context = [
    "OpenAI provides powerful tools for developers.",
    "AI is revolutionizing technology.",
    "Developers use OpenAI tools for creating applications.",
]
question = "What does OpenAI provide?"

# Embeddings for context sentences and question
context_embeddings = [flexiai.embedding_manager.create_embeddings(sentence) for sentence in context]
question_embedding = flexiai.embedding_manager.create_embeddings(question)

# Find the most relevant context sentence
similarities = [cosine_similarity(question_embedding, emb) for emb in context_embeddings]
most_relevant_index = np.argmax(similarities)
print(f"Answer: {context[most_relevant_index]}")


Answer: OpenAI provides powerful tools for developers.


### 6. **Sentiment Analysis**

#### Example: Sentiment Classification with Logistic Regression
Using embeddings as features for training a sentiment classifier.

In [9]:
# Sample data
texts = [
    "I love using OpenAI tools.",
    "AI is revolutionizing the industry.",
    "I found a great recipe for pasta.",
    "Artificial intelligence is fascinating.",
    "I am so frustrated with this service.",
    "This is the worst experience I've ever had.",
]
labels = [1, 1, 1, 1, 0, 0]  # 1 for positive, 0 for negative

# Generate embeddings for texts using FlexiAI `create_embeddingss` function
embeddings = [flexiai.embedding_manager.create_embeddings(text) for text in texts]

# Filter out any None values if embedding generation failed for any text
embeddings = [emb for emb in embeddings if emb is not None]
labels = [label for emb, label in zip(embeddings, labels) if emb is not None]

# Split the data into training and test sets
train_embeddings, test_embeddings, train_labels, test_labels = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Train the logistic regression classifier
classifier = LogisticRegression()
classifier.fit(train_embeddings, train_labels)

# Evaluate the classifier
predicted_test_labels = classifier.predict(test_embeddings)
accuracy = accuracy_score(test_labels, predicted_test_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Classify new texts
new_texts = [
    "OpenAI's tools are amazing!",
    "I am very disappointed with the service.",
]
# Generate NEW embeddings for texts using FlexiAI `create_embeddingss` function
new_embeddings = [flexiai.embedding_manager.create_embeddings(text) for text in new_texts]
predicted_labels = classifier.predict(new_embeddings)

for text, label in zip(new_texts, predicted_labels):
    sentiment = 'positive' if label == 1 else 'negative'
    print(f"Text: '{text}' - Sentiment: {sentiment}")


Test Accuracy: 100.00%
Text: 'OpenAI's tools are amazing!' - Sentiment: positive
Text: 'I am very disappointed with the service.' - Sentiment: negative


### 7. **Enhanced Sentiment Analysis with Data Augmentation**

#### Example: Sentiment Classification with Logistic Regression and Data Augmentation
In this example, we improve sentiment classification by augmenting the data to create more variations of the texts. This helps in better generalization of the classifier.

In [10]:
# Sample data
texts = [
    "I love using OpenAI tools.",
    "AI is revolutionizing the industry.",
    "I found a great recipe for pasta.",
    "Artificial intelligence is fascinating.",
    "I am so frustrated with this service.",
    "This is the worst experience I've ever had.",
    "I hate waiting in long lines.",
    "The product quality is terrible.",
    "The new feature is awesome!",
    "I am very pleased with the customer support.",
]
labels = [1, 1, 1, 1, 0, 0, 0, 0, 1, 1]  # 1 for positive, 0 for negative

# Data augmentation: create slight variations of the texts
augmented_texts = texts + [
    "I absolutely love using OpenAI tools.",
    "The industry is being revolutionized by AI.",
    "I discovered a fantastic recipe for pasta.",
    "AI is incredibly fascinating.",
    "This service is extremely frustrating.",
    "I've never had a worse experience.",
    "Waiting in long lines is so annoying.",
    "The quality of the product is horrible.",
    "The new feature is really awesome!",
    "Customer support has been very pleasing.",
    "Using OpenAI tools is such a pleasure.",
    "AI is changing the world.",
    "Found an amazing pasta recipe!",
    "I find artificial intelligence really interesting.",
    "This service makes me so angry.",
    "Worst service experience ever.",
    "I despise long wait times.",
    "Product quality is very poor.",
    "Loving the new feature!",
    "Very satisfied with customer support.",
]
augmented_labels = labels + labels + labels

# Generate embeddings for texts
embeddings = [flexiai.embedding_manager.create_embeddings(text) for text in augmented_texts]

# Filter out any None values if embedding generation failed for any text
embeddings = [emb for emb in embeddings if emb is not None]
augmented_labels = [label for emb, label in zip(embeddings, augmented_labels) if emb is not None]

# Split the data into training and test sets
train_embeddings, test_embeddings, train_labels, test_labels = train_test_split(embeddings, augmented_labels, test_size=0.2, random_state=42, stratify=augmented_labels)

# Train the logistic regression classifier with regularization
classifier = LogisticRegression(C=1.0, solver='liblinear')
classifier.fit(train_embeddings, train_labels)

# Cross-validation using StratifiedKFold to ensure balanced splits
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(classifier, train_embeddings, train_labels, cv=cv)

print(f"Cross-Validation Accuracy: {cv_scores.mean() * 100:.2f}%")

# Evaluate the classifier on the test set
predicted_test_labels = classifier.predict(test_embeddings)
accuracy = accuracy_score(test_labels, predicted_test_labels)
precision = precision_score(test_labels, predicted_test_labels)
recall = recall_score(test_labels, predicted_test_labels)
f1 = f1_score(test_labels, predicted_test_labels)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")

# Classify new texts
new_texts = [
    "OpenAI's tools are amazing!",
    "I am very disappointed with the service.",
]
new_embeddings = [flexiai.embedding_manager.create_embeddings(text) for text in new_texts]
predicted_labels = classifier.predict(new_embeddings)

for text, label in zip(new_texts, predicted_labels):
    sentiment = 'positive' if label == 1 else 'negative'
    print(f"Text: '{text}' - Sentiment: {sentiment}")


Cross-Validation Accuracy: 96.00%
Test Accuracy: 100.00%
Precision: 100.00%
Recall: 100.00%
F1 Score: 100.00%
Text: 'OpenAI's tools are amazing!' - Sentiment: positive
Text: 'I am very disappointed with the service.' - Sentiment: negative


In [11]:
# Print embeddings and labels for inspection
print("Embeddings and Labels:")
for emb, label in zip(embeddings, augmented_labels):
    print(f"Label: {label}, Embedding: {emb[:5]}...")

Embeddings and Labels:
Label: 1, Embedding: [-0.02184969 -0.02521532 -0.00558695 -0.03346786  0.00782846]...
Label: 1, Embedding: [-0.01700989 -0.02424673 -0.00913568 -0.00095357 -0.00373129]...
Label: 1, Embedding: [ 0.01686893  0.00550202  0.00944137 -0.00730333  0.01237709]...
Label: 1, Embedding: [-0.0104941  -0.00198581  0.01232169 -0.01587353  0.01007433]...
Label: 0, Embedding: [-0.02796874 -0.00581143 -0.02613494 -0.00895791 -0.00475601]...
Label: 0, Embedding: [-0.02066449 -0.01416779  0.01079451 -0.02041462 -0.03453244]...
Label: 0, Embedding: [-2.92321499e-02  2.11152546e-05  1.04639269e-02 -1.25785656e-02
 -3.16231698e-02]...
Label: 0, Embedding: [-0.00999594  0.00082101 -0.01090165 -0.00213042 -0.01635578]...
Label: 1, Embedding: [-0.01405413  0.01471457  0.00048418 -0.03320684 -0.01682797]...
Label: 1, Embedding: [-0.0072394  -0.00262535 -0.00274349 -0.03956399 -0.0264635 ]...
Label: 1, Embedding: [-0.02164579 -0.02529381 -0.00410234 -0.03882243  0.00586577]...
Label: 1, 